In [34]:
import cv2
import pyrealsense2 as rs
import numpy as np

In [35]:
dataset_path = '..\\..\\Dataset'

# Setup (Realsense)

In [36]:
def read_bag_file(file_name, real_time=False):
    pipe = rs.pipeline()
    cfg = rs.config()
    cfg.enable_device_from_file(file_name, repeat_playback=False)
    profile = pipe.start(cfg)
    playback = profile.get_device().as_playback()
    playback.set_real_time(real_time) # False: no frame drop
    
    # Get the frame shape of the color sensor
    frames = pipe.wait_for_frames()
    color_frame = frames.get_color_frame()
    frame_shape = (color_frame.get_height(), color_frame.get_width())

    return pipe, cfg, profile, playback, frame_shape

In [37]:
# Read the bag file
bag_file = dataset_path + "\\me_and_sasa.bag"

pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
duration = playback.get_duration()
print("Video duration: ", duration)
print("Frame shape: ", frame_shape)

Video duration:  0:00:43.503743
Frame shape:  (480, 848)


# Visualize the data

In [38]:
def post_process_depth_frame(depth_frame, min_distance=0, max_distance=1.5, decimation_magnitude = 1.0, spatial_magnitude = 2.0, spatial_smooth_alpha = 0.5, spatial_smooth_delta = 20, temporal_smooth_alpha = 0.4, temporal_smooth_delta = 20):
    # Post processing possible only on the depth_frame
    assert (depth_frame.is_depth_frame())

    # Available filters
    decimation_filter = rs.decimation_filter()
    threshold_filter = rs.threshold_filter()
    depth_to_disparity = rs.disparity_transform(True)
    spatial_filter = rs.spatial_filter()
    temporal_filter = rs.temporal_filter()
    disparity_to_depth = rs.disparity_transform(False)
    hole_filling = rs.hole_filling_filter(1) # https://intelrealsense.github.io/librealsense/doxygen/classrs2_1_1hole__filling__filter.html

    # Apply the control parameters for the filters
    decimation_filter.set_option(rs.option.filter_magnitude, decimation_magnitude)
    threshold_filter.set_option(rs.option.min_distance, min_distance)
    threshold_filter.set_option(rs.option.max_distance, max_distance)
    spatial_filter.set_option(rs.option.filter_magnitude, spatial_magnitude)
    spatial_filter.set_option(rs.option.filter_smooth_alpha, spatial_smooth_alpha)
    spatial_filter.set_option(rs.option.filter_smooth_delta, spatial_smooth_delta)
    temporal_filter.set_option(rs.option.filter_smooth_alpha, temporal_smooth_alpha)
    temporal_filter.set_option(rs.option.filter_smooth_delta, temporal_smooth_delta)

    # Apply the filters
    # Post processing order : https://dev.intelrealsense.com/docs/post-processing-filters
    # Depth Frame >> Decimation Filter >> Depth2Disparity Transform >> Spatial Filter >> Temporal Filter >> Disparity2Depth Transform >> Hole Filling Filter >> Filtered Depth
    filtered_frame = decimation_filter.process(depth_frame)
    filtered_frame = threshold_filter.process(filtered_frame)
    filtered_frame = depth_to_disparity.process(filtered_frame)
    filtered_frame = spatial_filter.process(filtered_frame)
    filtered_frame = temporal_filter.process(filtered_frame)
    filtered_frame = disparity_to_depth.process(filtered_frame)
    filtered_frame = hole_filling.process(filtered_frame)
    
    # Cast to depth_frame so that we can use the get_distance method afterwards
    depth_frame_filtered = filtered_frame.as_depth_frame()

    return depth_frame_filtered

In [39]:
colorizer = rs.colorizer()

align = rs.align(rs.stream.color)

# Read the full stream
pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
num_frames = 0
wait_key = 1

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        depth_frame = post_process_depth_frame(aligned_depth_frame)

        # Colorize the depth frame
        depth_color_frame = colorizer.colorize(depth_frame)

        # Convert frames to images
        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        cv2.imshow("Color Image", color_image)
        cv2.imshow("Depth Image", depth_color_image)

        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        """ # Press s to write the images on disk
        if key == ord('s'):
            cv2.imwrite(".\\images\\color_images\\images\\color_" + str(num_frames) + ".png", color_image)
            cv2.imwrite(".\\images\\depth_images\\depth_" + str(num_frames) + ".png", depth_color_image) """

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1

# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop()

print("Total number of frames: ", num_frames)

Total number of frames:  268


# Extract height of the participants

In [7]:
# # Create a mask to keep ony the ROI, under the camera
# mask = np.zeros(frame_shape, dtype=np.uint8)
# center_y, center_x = frame_shape[0] // 2, frame_shape[1] // 2

# # Mask size
# half_height = frame_shape[0] // 3
# top_left = (0, half_height)
# bottom_right = (848, 480)

# cv2.rectangle(mask, top_left, bottom_right, 255, thickness=-1)

# plt.imshow(mask, cmap='gray')
# plt.show()

In [40]:
# Setup the aruco detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_MIP_36h12)
aruco_params = cv2.aruco.DetectorParameters()

aruco_params.errorCorrectionRate = 0.2 # default 0.6
aruco_params.polygonalApproxAccuracyRate = 0.05 # default 0.03

detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params) 

In [41]:
camera_height = 2.568 # meters, ground truth

pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
num_frames = 0
wait_key = 1

# Data structure to store the a list of height for every aruco marker id
heights = {}

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        depth_frame = post_process_depth_frame(aligned_depth_frame)

        # Colorize the depth frame
        depth_color_frame = colorizer.colorize(depth_frame)

        # Convert frames to images
        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(color_image)
        output_image = depth_color_image.copy()
        cv2.aruco.drawDetectedMarkers(output_image, corners, ids)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Calculate the distance using the center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)
            distance = depth_frame.get_distance(x, y)

            # Calculate the height
            height = camera_height - distance

            # Display the height with an offset to avoid overlap
            text_position_y = 30 + k * 40
            cv2.putText(output_image, "ID: {} Height: {:.2f}m".format(id, height), (10, text_position_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Store the height for the aruco marker id
            if not id in heights:
                heights[id] = []
                
            heights[id].append(height)

        cv2.imshow("Output", output_image)
        cv2.imshow("Color Image", color_image)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop()

In [9]:
def remove_outliers(data, m=2):
    # Z-score method
    data = np.array(data)
    mean = np.mean(data)
    std = np.std(data)
    filtered_data = [x for x in data if abs(x - mean) / std < m]
    return filtered_data

def calculate_mean_height(heights):
    mean_heights = {}

    for key, height_list in heights.items():
        filtered_heights = remove_outliers(height_list)
        if filtered_heights:
            mean_heights[key] = np.mean(filtered_heights)

    return mean_heights

# Remove outliers and calculate the mean height for each aruco marker id
mean_heights = calculate_mean_height(heights)
print(mean_heights)

{88: 1.727318140961907, 24: 1.824235255914576}


# Projection of the point on the floor

In [10]:
# Matching points on the floor, 130cm and 170cm
points_floor = np.array([
    [1042, 737, 0],
    [1161, 747, 0],
    [1287, 760, 0],
    [1405, 771, 0],
    [1025, 861, 0],
    [1148, 870, 0],
    [1274, 882, 0],
    [1397, 891, 0],
    [1011, 986, 0],
    [1138, 997, 0],
    [1265, 1007, 0],
    [1392, 1018, 0],
    [996, 1111, 0],
    [1128, 1120, 0],
    [1257, 1128, 0],
    [1382, 1134, 0]
])

points_130 = np.array([
    [805, 395, 130],
    [1060, 420, 130],
    [1320, 449, 130],
    [1554, 473, 130],
    [777, 651, 130],
    [1040, 677, 130],
    [1301, 702, 130],
    [1549, 731, 130],
    [746, 914, 130],
    [1024, 937, 130],
    [1286, 960, 130],
    [1532, 976, 130],
    [722, 1130, 130],
    [1017, 1178, 130],
    [1257, 1179, 130],
    [1507, 1208, 130]
])

points_170 = np.array([
    [598, 91, 170],
    [970, 118, 170],
    [1346, 167, 170],
    [1693, 206, 170],
    [537, 445, 170],
    [931, 489, 170],
    [1323, 536, 170],
    [1685, 576, 170],
    [481, 842, 170],
    [906, 873, 170],
    [1301, 916, 170],
    [1664, 933, 170],
    [444, 1172, 170],
    [895, 1240, 170],
    [1261, 1241, 170],
    [1634, 1278, 170]
])

In [11]:
image_path = dataset_path + '\\SINGLE\\Image__2024-03-20__16-55-35.png'
image = cv2.imread(image_path)

for pf, p130, p170 in zip(points_floor, points_130, points_170):
    cv2.line(image, pf[:2], p130[:2], (255, 0, 0), 1)
    cv2.line(image, p130[:2], p170[:2], (0, 255, 0), 1)

cv2.namedWindow('Projection Lines', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Lines', image)
cv2.resizeWindow('Projection Lines', 864, 648)
cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
# Compute the 3 homographies
points_floor_2d = points_floor[:, :2]
points_130_2d = points_130[:, :2]
points_170_2d = points_170[:, :2]

H_0, _ = cv2.findHomography(points_floor_2d, points_floor_2d) # Identity matrix (hopefully :))
H_130, _ = cv2.findHomography(points_130_2d, points_floor_2d)
H_170, _ = cv2.findHomography(points_170_2d, points_floor_2d)

# Interpolate the homography for a given height, between 0 and 210 cm
def interpolate_homography(z, H_0, H_130, H_170):
    if z <= 130: # linear interpolation between 0 and 130
        alpha = z / 130
        return (1 - alpha) * H_0 + alpha * H_130
    
    elif z <= 170: # linear interpolation between 130 and 170
        alpha = (z - 130) / 40 # /40 to normalize the alpha value (130 < z <= 170)
        return (1 - alpha) * H_130 + alpha * H_170
    
    else: # extrapolation between 170 and 210
        # Basically, from 170 cm, the transformation continues to change in the same way as it did between 130 cm and 170 cm
        H_210 =  H_170 + (H_170 - H_130)
        alpha = (z - 170) / 40 # /40 to normalize the alpha value (170 < z <= 210)
        return (1 - alpha) * H_170 + alpha * H_210

# Project a point on the floor, given its 3D coordinates
def project_point(point, z):
    H = interpolate_homography(z, H_0, H_130, H_170)
    point_homogeneous = np.hstack([point, 1])
    projected_point_homogeneous = H @ point_homogeneous
    projected_point = projected_point_homogeneous[:2] / projected_point_homogeneous[2]
    return projected_point

In [13]:
# Small example to test the homography interpolation
image = cv2.imread(image_path)

def project_points(points, H):
    points_homogeneous = np.hstack([points[:, :2], np.ones((points.shape[0], 1))])
    projected_points_homogeneous = (H @ points_homogeneous.T).T
    projected_points = projected_points_homogeneous[:, :2] / projected_points_homogeneous[:, 2][:, np.newaxis]
    return projected_points

cv2.namedWindow('Projection Points', cv2.WINDOW_NORMAL)
cv2.imshow('Projection Points', image)
cv2.resizeWindow('Projection Points', 864, 648)

# Draw the points at 170 cm
for point in points_170.astype(int):
    cv2.circle(image, tuple(point[:2]), 10, (255, 0, 0), -1)

def on_trackbar(z):
    z = int(z)
    H = interpolate_homography(z, H_0, H_130, H_170)
    projected_points = project_points(points_170, H)
    image_with_points = image.copy()
    for point in projected_points.astype(int):
        cv2.circle(image_with_points, tuple(point), 10, (0, 0, 255), -1)
    cv2.imshow('Projection Points', image_with_points)

cv2.createTrackbar('Height', 'Projection Points', 170, 210, on_trackbar)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
baslar_file = dataset_path + '\\solo_experiment\\7__40069753__20240625_144414557.mp4'
cap = cv2.VideoCapture(baslar_file)
wait_key = 1
height = 180
projected_points = []

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(frame)
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for k in range(len(corners)):
            c = corners[k][0]

            # Center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)

            # Project the point on the floor
            point = (x, y)
            projected_point = project_point(point, height)
            projected_points.append(projected_point)

            cv2.circle(frame, point, 10, (0, 0, 255), -1)
        
        # Draw all the projected points to get the trajectory
        for projected_point in projected_points:
            point = projected_point.astype(int)
            cv2.circle(frame, tuple(point), 10, (0, 255, 0), -1)

        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        cv2.imshow('Frame', frame)
        cv2.resizeWindow('Frame', 864, 648)
        
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0
    else:
        break

cap.release()
cv2.destroyAllWindows()


# Estimate exit times

## 1. Estimate heights

In [25]:
camera_height = 2.568 # meters, ground truth

bag_file = dataset_path + '\\solo_experiment\\rec.bag'
pipe, cfg, profile, playback, frame_shape = read_bag_file(bag_file)
num_frames = 0

# Data structure to store the a list of height for every aruco marker id
heights = {}

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        depth_frame = post_process_depth_frame(aligned_depth_frame)

        # Convert color frame to image
        color_image = np.asanyarray(color_frame.get_data())

        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(color_image)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Calculate the distance using the center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)
            distance = depth_frame.get_distance(x, y)

            # Calculate the height
            height = camera_height - distance

            # Store the height for the aruco marker id
            if not id in heights:
                heights[id] = []
                
            heights[id].append(height)

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    pipe.stop()

print("Total number of frames: ", num_frames)

# Remove outliers and calculate the mean height for each aruco marker id
mean_heights = calculate_mean_height(heights)
print(mean_heights)

Stream ended
Total number of frames:  916
{17: 1.8451363312547855}


## 2. 

In [30]:
# Read the baslar video file
baslar_file = dataset_path + '\\solo_experiment\\7__40069753__20240625_144414557.mp4'
cap = cv2.VideoCapture(baslar_file)
wait_key = 1
num_frames = 0

# Dictionary to store the projected points for each aruco marker id, and the color for each id, only for visualization
projected_points = {}
color_points = {}
for id, _ in heights.items():
    color_points[id] = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))

# Data structure to store the enter and exit frame for each aruco marker id
enter_frame = {}
exit_frame = {}
for id, _ in heights.items():
    enter_frame[id] = -1
    exit_frame[id] = -1

# Y coordinate of the "exit line"
y_exit = 1130

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(frame)
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)

            # Project the point on the floor
            point = (x, y)
            height = mean_heights[id] * 100 # cm
            projected_point = project_point(point, height)

            # Store the projected point for the aruco marker id
            if not id in projected_points:
                projected_points[id] = []
                
            projected_points[id].append(projected_point)

            # Update the enter and exit frame
            if enter_frame[id] == -1:
                enter_frame[id] = num_frames

            if exit_frame[id] == -1 and y > y_exit:
                exit_frame[id] = num_frames

            # Draw a circle at the center of the aruco marker
            cv2.circle(frame, point, 10, (0, 0, 255), -1)
        
        # Draw all the projected points to get the trajectory
        for id, points in projected_points.items():
            for projected_point in points:
                point = projected_point.astype(int)
                if point[1] < y_exit:
                    cv2.circle(frame, tuple(point), 10, color_points[id], -1)

        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        cv2.imshow('Frame', frame)
        cv2.resizeWindow('Frame', 864, 648)
        
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1

    else:
        break

cap.release()
cv2.destroyAllWindows()


In [33]:
print("Enter frames: ", enter_frame)
print("Exit frames: ", exit_frame)

# Compute the difference between the enter and exit frame, to estimate the exit time for each id
exit_time = {}
fps = 30
for id, enter in enter_frame.items():
    exit = exit_frame[id]
    if enter != -1 and exit != -1:
        exit_time[id] = (exit - enter) / fps

# Print exit times in seconds
for id, time in exit_time.items():
    print("ID: {} Exit time: {:.2f}s".format(id, time))

Enter frames:  {17: 183}
Exit frames:  {17: 210}
ID: 17 Exit time: 0.90s


# Attemping to enhance Aruco codes

In [ ]:
""" camera_height = 2.65 # meters => NOT SURE ABOUT THIS VALUE

pipe, cfg, profile, playback, frame_shape = read_bag_file(file_name)
num_frames = 0
wait_key = 1

# Empty the distance_images folder
files = glob.glob('distance_images/*')
for f in files:
    os.remove(f)

# Data structure to store the a list of height for every aruco code id
heights = {}

try:
    while True:
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame so they have the same shape
        aligned_depth, aligned_color = align_frames(frames)
        color_image = np.asanyarray(aligned_color.get_data())
        depth_color_frame = colorizer.colorize(aligned_depth)
        depth_color_image = np.asanyarray(depth_color_frame.get_data())

        # Apply the mask to the depth image
        depth_color_image = cv2.bitwise_and(depth_color_image, depth_color_image, mask=mask)

        # Convert the image to HSV color space
        hsv_image = cv2.cvtColor(depth_color_image, cv2.COLOR_BGR2HSV)

        # Define the range for red color in HSV
        # Note: Red can appear in two ranges in HSV
        lower_red1 = np.array([0, 70, 50])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([170, 70, 50])
        upper_red2 = np.array([180, 255, 255])

        # Create masks for red color
        red_mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
        red_mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)

        # Combine the two masks
        red_mask = cv2.bitwise_or(red_mask1, red_mask2)

        # Apply the mask to the original image
        result = cv2.bitwise_and(depth_color_image, depth_color_image, mask=red_mask)

        # Convert the result to grayscale and apply thresholding
        gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        _, thresh_result = cv2.threshold(gray_result, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply the closing morphological operation to fill in holes
        kernel = np.ones((30, 30), np.uint8)
        closed_result = cv2.morphologyEx(thresh_result, cv2.MORPH_CLOSE, kernel)

        # Find contours of the red square
        contours, _ = cv2.findContours(closed_result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create a mask to keep only the aruco code in the color image
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 100*100 and area < 200*200:
                x, y, w, h = cv2.boundingRect(contour)  
                arcuo_code_mask = np.zeros(frame_shape, dtype=np.uint8)
                
                # Expand the bounding box by 50 pixels
                sz = (w + h) // 2
                x1 = max(0, x - 25)
                y1 = max(0, y - 25)
                x2 = min(640, x + sz + 25)
                y2 = min(480, y + sz + 25)
                cv2.rectangle(arcuo_code_mask, (x1, y1), (x2, y2), (255, 255, 255), -1)
                color_image = cv2.bitwise_and(color_image, color_image, mask=arcuo_code_mask)

                cv2.imwrite("color_images\motion_blur_mask\motion_blur" + str(num_frames) + ".png", color_image)

        cv2.imshow("Output", color_image)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop() """

' camera_height = 2.65 # meters => NOT SURE ABOUT THIS VALUE\n\npipe, cfg, profile, playback, frame_shape = read_bag_file(file_name)\nnum_frames = 0\nwait_key = 1\n\n# Empty the distance_images folder\nfiles = glob.glob(\'distance_images/*\')\nfor f in files:\n    os.remove(f)\n\n# Data structure to store the a list of height for every aruco code id\nheights = {}\n\ntry:\n    while True:\n        frames = pipe.wait_for_frames()\n\n        # Align the depth frame to color frame so they have the same shape\n        aligned_depth, aligned_color = align_frames(frames)\n        color_image = np.asanyarray(aligned_color.get_data())\n        depth_color_frame = colorizer.colorize(aligned_depth)\n        depth_color_image = np.asanyarray(depth_color_frame.get_data())\n\n        # Apply the mask to the depth image\n        depth_color_image = cv2.bitwise_and(depth_color_image, depth_color_image, mask=mask)\n\n        # Convert the image to HSV color space\n        hsv_image = cv2.cvtColor(depth

In [ ]:
""" image = cv2.imread("color_images\motion_blur\motion_blur1229.png", cv2.IMREAD_GRAYSCALE)
corners, ids, rejected = detector.detectMarkers(image)
output_image = image.copy()
if corners is not None:
    print("Detected markers: ", len(corners))

cv2.aruco.drawDetectedMarkers(output_image, corners, ids)
cv2.imshow("Output", output_image)

# Show the marker id image
marker_id = ids[0][0]
print("Marker ID: ", marker_id)
marker_size = 200  # Taille de l'image générée
aruco_marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)
cv2.imshow("Marker", aruco_marker_image)
cv2.waitKey(0)
cv2.destroyAllWindows() """

' image = cv2.imread("color_images\\motion_blur\\motion_blur1229.png", cv2.IMREAD_GRAYSCALE)\ncorners, ids, rejected = detector.detectMarkers(image)\noutput_image = image.copy()\nif corners is not None:\n    print("Detected markers: ", len(corners))\n\ncv2.aruco.drawDetectedMarkers(output_image, corners, ids)\ncv2.imshow("Output", output_image)\n\n# Show the marker id image\nmarker_id = ids[0][0]\nprint("Marker ID: ", marker_id)\nmarker_size = 200  # Taille de l\'image générée\naruco_marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)\ncv2.imshow("Marker", aruco_marker_image)\ncv2.waitKey(0)\ncv2.destroyAllWindows() '